### Wczytanie tweetów

In [ ]:
import pandas as pd
import pymongo

DB_CONN_STRING = "mongodb://localhost:27017/"
DB_NAME = "recent-poland-covid-vaccine-tweets"
COLLECTION_NAME = "tweets"

mongo_client = pymongo.MongoClient(DB_CONN_STRING)
database_name = mongo_client[DB_NAME]
data_collection = database_name[COLLECTION_NAME]

projection = {"_id": 1, "conversationId":1, "sentiment": 1}
tweets = [tweet for tweet in data_collection.find({}, projection)]

df = pd.DataFrame(tweets)
# df

### Reorganizacja danych

In [ ]:
import numpy as np

In [ ]:
roots = df.query('_id == conversationId')["_id"].values
df["root_exists"] = df.apply(lambda row: row["conversationId"] in roots, axis=1)
df = df.query('root_exists == True')

In [ ]:
def is_positive(sentiment_val):
    if sentiment_val["_id"] == sentiment_val["conversationId"]:
        return 0
    sentiment_val = float(sentiment_val['sentiment'])
    return 1 if sentiment_val > 0 else 0

def is_neutral(sentiment_val):
    if sentiment_val["_id"] == sentiment_val["conversationId"]:
        return 0
    sentiment_val = float(sentiment_val['sentiment'])
    return 1 if sentiment_val == 0 else 0

def is_negative(sentiment_val):
    if sentiment_val["_id"] == sentiment_val["conversationId"]:
        return 0
    sentiment_val = float(sentiment_val['sentiment'])
    return 1 if sentiment_val < 0 else 0

df["positive"] = df.apply(is_positive, axis=1)
df["neutral"] = df.apply(is_neutral, axis=1)
df["negative"] = df.apply(is_negative, axis=1)
df

In [ ]:
df.index = df['_id']
df["conversationRootSentiment"] = df.apply(lambda row: df.loc[row['conversationId']]['sentiment'], axis=1)

In [ ]:
sentiment_grouped_df = df[['conversationRootSentiment', 'positive', 'neutral', 'negative']].groupby(by=['conversationRootSentiment']).sum()
sentiment_grouped_df

In [ ]:
def get_sentiment(sentiment_val):
    sentiment_val = float(sentiment_val)
    if sentiment_val > 0:
        return "positive"
    if sentiment_val == 0:
        return "neutral"
    if sentiment_val < 0:
        return "negative"


sentiment_grouped_df["conversationRootSentiment"] = sentiment_grouped_df.index
sentiment_grouped_df["conversationRootSentiment2"] = sentiment_grouped_df.apply(lambda row: get_sentiment(row["conversationRootSentiment"]), axis=1)
sentiment_grouped_df2 = sentiment_grouped_df.groupby(by=['conversationRootSentiment2']).sum()

In [ ]:
sentiment_grouped_df2_norm = sentiment_grouped_df2.div(sentiment_grouped_df2.sum(axis=1), axis=0)
sentiment_grouped_df2_norm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(15,10))
ax = sns.heatmap(sentiment_grouped_df2_norm, annot=True)